In [111]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [112]:
offers_connection = sqlite3.connect('../datasets/offers_dataset.db')
offers_frame = pd.read_sql_query('''SELECT * FROM offers''', offers_connection)
offers_connection.close()

# [!] Le descrizioni all'interno del notebook non sono mostrate nella loro versione completa
#     per accedere ad una specifica descrizione usare il metodo: offers_frame.loc[ID, NomeColonna]
offers_frame

,Name,Description,Location
0,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria"
1,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona
2,Software Developer,We are looking for talented and passionate peo...,55100 Lucca
3,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata"
4,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia"
...,...,...,...
130,Sviluppatori Web Mobile,Stiamo cercando 2 Sviluppatori web/mobile con ...,33170 Pordenone
131,FrontEnd Javascript,Luogo di lavoroMilanoInizioImmediato – Durata:...,"Milano, Lombardia"
132,Programmatore PHP Junior,Descrizione del Ruolo: \n Shaggy Owl s.r.l.s....,01033 Civita Castellana
133,Full Stack Web Developer,la risorsa da inserire nel contesto aziendale ...,20123 Milano


## Data Exploration
Come prima cosa esaminiamo il nostro dataset di offerte, in particolare dovremmo verificare la qualità dei dati:
- Base   
    - Presenza di valori nulli
    - (**Importante**) Presenza di sinonimi geografici
    - Presenza di informazioni utili
- Avanzato
    - Analisi delle Parole Chiave (es. Titoli)
    - Analisi della distribuzione dei dati

**Presenza di Valori Nulli e Duplicati**
Come è possibile vedere di seguito il dataset contiene valori nulli solo per la colonna Location, il che però è normale, in quanto non tutte le Offerte di lavoro sono in presenza, quindi le Location mancanti rappresentano il numero di Offerte di lavoro in Remote.
Questa informazione è importante perchè in seguito ci sarà la necessità di rappresentare queste informazioni in modo omogeneo, in modo tale che un modello il cui scopo è raggruppare le Offerte (clustering, come proposto nell'Analisi del Problema) possa farlo in modo adeguato.

In [113]:
offers_frame.drop_duplicates(inplace=True)
offers_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129 entries, 0 to 134
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         129 non-null    object
 1   Description  129 non-null    object
 2   Location     102 non-null    object
dtypes: object(3)
memory usage: 4.0+ KB


## Presenza di Sinonimi Geografici
Il dataset contiene attualmente molteplici istanze in cui la stessa Location è riportata in modi diversi, come \<Cap>, \<Città> e \<Città>, \<Regione>.


In [114]:
#example_city = "Bologna"

#onsite_offers = offers_frame[offers_frame["Location"].notna()]
#geo_syn = onsite_offers.Location.map(lambda text: example_city in text)
#onsite_offers[geo_syn]

**Analisi dei Titoli**
Di seguito vengono analizzati i Titoli delle Offerte di lavoro con lo scopo di verificare la presenza di termini comuni, in particolare vengono usate delle basilari tecniche di Natural Language Processing come:
- **Tokenization**, utilizzata per dividere frasi o documenti in parole (*title.split()*) 
- **Rimozione della Punteggiatura**, utilizzata per migliorare la qualità dei dati estratti, in modo da non influenzare i risultati 
- **Normalizzazione**, utilizzata per ridurre la complessità del vocabolario e trattare parole con maiuscole e minuscole come equivalenti (*t.lower()*)

In [115]:
from string import punctuation
from collections import Counter

def count_words(document: str, counter: Counter):
    """
    :param document: a string representation of the document to inspect
    :param counter: a Python built-in object used to count occurrences of items
    """
    punct = [p for p in punctuation]
    for t in document.split():
        if not t in punct:
            counter[t.lower()] += 1


titles_counts = Counter()
offers_frame.Name.map(lambda text: count_words(text, titles_counts))

titles_counts

Counter({'software': 42,
         'developer': 42,
         'engineer': 32,
         'network': 21,
         'web': 15,
         'junior': 13,
         'intern': 9,
         'senior': 8,
         'frontend': 8,
         'sviluppatore': 8,
         '2024': 8,
         'full': 6,
         'development': 5,
         'engineering': 5,
         'graduate': 5,
         'security': 5,
         'programmatore': 4,
         'stack': 4,
         'manager': 4,
         'java': 3,
         'c#': 3,
         'apprenticeship': 3,
         '(london)': 3,
         'remote)': 3,
         'summer': 3,
         'analyst': 3,
         'internship': 3,
         '–': 3,
         'specialist': 3,
         'stage': 2,
         'settore': 2,
         'sanità': 2,
         'front-end': 2,
         'app': 2,
         'remote': 2,
         'uk': 2,
         'technology': 2,
         'apprentice': 2,
         'front': 2,
         'end': 2,
         '(100%': 2,
         'lead': 2,
         'system': 2,
         'en

Analisi delle occorrenze di parole:

In [116]:
# Jacopo plotta qui

### Skill Extraction
Per procedere con l'estrazione delle Competenze dalle Offerte per prima cosa importiamo il relativo dataset.   

In [117]:
with sqlite3.connect('../datasets/skills_dataset.db') as skills_connection:
    skills_frame = pd.read_sql_query('SELECT * FROM skills', skills_connection)
    skills_frame.set_index('ID', inplace=True)
    
skills_frame

,SKILL,TYPE
ID,,
0,JavaScript,Programming Language
1,HTML,Programming Language
2,Python,Programming Language
3,SQL,Programming Language
4,TypeScript,Programming Language
...,...,...
95,APT,Tool
96,Unity 3D,Tool
97,Pacman,Tool


In [118]:
# Get Skill list
skills_list = [skill.lower() for skill in skills_frame['SKILL'].tolist()]

# Symbols to remove
punct = [p for p in punctuation]
punct.remove('+')
punct.remove('#')

removal = {p: '' for p in punct}
removal['\n'] = ''

def remove_symbols(description: str, remove_map: dict) -> str:
    for old, new in remove_map.items():
        description = description.replace(old, new)
    return description.lower()


def extract_symbols(description: str, available_symbols: list) -> set:
    s = set()
    for word in description.split():
        if word in available_symbols:
            s.add(word)
    return s 

In [123]:
with_no_skills = 0
for i, offer_description in enumerate(offers_frame.loc[:, 'Description']):
    desc = remove_symbols(offer_description, removal)
    offer_skills = extract_symbols(desc, skills_list)
    if not offer_skills:
        print(f'{i}-Offer: No Skills')
        with_no_skills += 1
    else:
        print(offer_skills)

print(f'There are {with_no_skills} Offers with no skills available')

0-Offer: No Skills
{'c', 'c#', 'java', 'c++', 'python'}
{'oracle', 'javascript', 'angular', 'java'}
{'c++', 'python'}
{'javascript', 'c#', 'sql', 'java', 'angularjs'}
{'java'}
6-Offer: No Skills
7-Offer: No Skills
{'javascript', 'html', 'css'}
{'mysql', 'java'}
{'jquery', 'javascript', 'c#', 'sql', 'html', 'css'}
{'java'}
{'docker', 'javascript', 'php', 'react', 'go', 'java', 'kubernetes'}
13-Offer: No Skills
{'angular', 'react', 'typescript'}
15-Offer: No Skills
16-Offer: No Skills
{'java'}
{'oracle', 'angular', 'react', 'java'}
19-Offer: No Skills
{'mysql', 'c#', 'sql', 'java', 'c++', 'python'}
{'oracle', 'javascript', 'typescript', 'flutter', 'react', 'angular', 'dart'}
{'c#', 'sql'}
{'javascript', 'php', 'sql', 'python'}
24-Offer: No Skills
{'dart', 'flutter'}
{'docker', 'vite', 'c#', 'sql'}
{'php', 'c#', 'java'}
{'python', 'c#', 'java'}
{'c++', 'c', 'java'}
{'docker', 'mysql', 'javascript', 'django', 'sql', 'css', 'postgresql', 'python'}
{'mysql', 'sql'}
32-Offer: No Skills
{'sql'

In [120]:
# Package necessari per la manipolazione di descrizioni in italiano (es. TF-IDF)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_words = set(stopwords.words('italian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anton\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Preparation
Per permettere il completo soddisfacimento dei requisiti, alcuni dati mancanti alle offerte dovranno essere aggiunti al dataset, tra cui:
- LocationType: il tipo di Location, se in remoto (Remote) o in sede (OnSite)
- Languages: le lingue naturali necessarie per l'offerta di lavoro

### Tipo di Location
Per estrarre il tipo di Location dalle informazioni attualmente già presenti nel dataset, 
saranno interpretate come di tipo Remote tutte le offerte che non presentano una Location.

In [121]:
LOCATION_TYPES = {"Remote" : "Remote", "On Site" : "OnSite"}
locations_present = offers_frame["Location"].notna()
types = []

for is_present in locations_present:
    if is_present:
        types.append(LOCATION_TYPES["On Site"])
    else:
        types.append(LOCATION_TYPES["Remote"])

offers_frame.loc[:, "LocationType"] = types

offers_frame

,Name,Description,Location,LocationType
0,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria",OnSite
1,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona,OnSite
2,Software Developer,We are looking for talented and passionate peo...,55100 Lucca,OnSite
3,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata",OnSite
4,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia",OnSite
...,...,...,...,...
130,Sviluppatori Web Mobile,Stiamo cercando 2 Sviluppatori web/mobile con ...,33170 Pordenone,OnSite
131,FrontEnd Javascript,Luogo di lavoroMilanoInizioImmediato – Durata:...,"Milano, Lombardia",OnSite
132,Programmatore PHP Junior,Descrizione del Ruolo: \n Shaggy Owl s.r.l.s....,01033 Civita Castellana,OnSite
133,Full Stack Web Developer,la risorsa da inserire nel contesto aziendale ...,20123 Milano,OnSite


### Linguaggi Richiesti
Per dedurre quali lingue parlate sono richieste da una data offerta di lavoro, viene utilizzato il modulo langdetect per identificare la lingua in cui è stata scritta la descrizione dell'offerta stessa, e successivamente tramite una ricerca all'interno del testo della descrizione per menzioni di ulteriori lingue.

In [122]:
from langdetect import detect
offers_langs = []
remove_map = {p:'' for p in punctuation}
supported_langs = ['italiano', 'inglese', 'italian', 'english']
eqs = [('italiano', 'italian'), ('inglese', 'english')]

for desc in offers_frame.Description:
    lang = detect(desc)
    if lang == 'it':
        lang = 'italiano'
    elif lang == 'en':
        lang = 'inglese'
    
    desc = remove_symbols(desc, remove_map)
    found = extract_symbols(desc, supported_langs)
    found.add(lang)
    found = list(found)
    
    for i, l in enumerate(found):
        for e in eqs:
            if l in e:
                found[i] = e[0]
                break
    
    offers_langs.append(found)
    
offers_frame.loc[:, "Languages"] = offers_langs

offers_frame

,Name,Description,Location,LocationType,Languages
0,Software Developer,Miniclip is a global leader in digital games w...,"Genova, Liguria",OnSite,"[inglese, inglese]"
1,Junior Software Developer,"NETtoWORK, azienda italiana nata nel 2016, ope...",17100 Savona,OnSite,[italiano]
2,Software Developer,We are looking for talented and passionate peo...,55100 Lucca,OnSite,"[inglese, inglese]"
3,Software Developer,ARESYS is a R&D oriented company with nearly ...,"Matera, Basilicata",OnSite,[inglese]
4,Senior Software Developer,Il/la Candidato/a dovrà padroneggiare: \n \n- ...,"Catania, Sicilia",OnSite,[italiano]
...,...,...,...,...,...
130,Sviluppatori Web Mobile,Stiamo cercando 2 Sviluppatori web/mobile con ...,33170 Pordenone,OnSite,[italiano]
131,FrontEnd Javascript,Luogo di lavoroMilanoInizioImmediato – Durata:...,"Milano, Lombardia",OnSite,[italiano]
132,Programmatore PHP Junior,Descrizione del Ruolo: \n Shaggy Owl s.r.l.s....,01033 Civita Castellana,OnSite,[italiano]
133,Full Stack Web Developer,la risorsa da inserire nel contesto aziendale ...,20123 Milano,OnSite,[italiano]
